In [ ]:
!pip install transformers torch

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertConfig, BertForPreTraining, BertTokenizerFast
from transformers import BertTokenizer, pipeline

In [ ]:
# 'ao_childes_curriculum.txt' contains your text data
with open('ao_childes_curriculum.txt', 'r', encoding='utf-8') as file:
    text_data = file.read()

In [ ]:
# Initialize the MiniLM-like tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased', model_max_length=512)

# Tokenize the text data
tokenized_data = tokenizer(text_data, return_tensors='pt', max_length=512, truncation=True)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
# custom dataset
class CustomDataset(Dataset):
    def __init__(self, tokenized_data):
        self.input_ids = tokenized_data['input_ids']
        self.attention_mask = tokenized_data['attention_mask']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
        }

# instance of the custom dataset
dataset = CustomDataset(tokenized_data)

In [ ]:
# Define the MiniLM-like configuration
config = BertConfig(
    vocab_size=tokenizer.vocab_size,
    hidden_size=256,
    num_hidden_layers=3,
    num_attention_heads=4,
)

In [ ]:
# Initialize the MiniLM-like model
model = BertForPreTraining(config=config)

# Set up the DataLoader
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# Training loop
num_epochs = 100
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

for epoch in range(num_epochs):
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask)
        prediction_logits = outputs.prediction_logits

        # Compute loss
        loss = criterion(prediction_logits.view(-1, tokenizer.vocab_size), input_ids.view(-1))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch + 1}, Batch loss: {loss.item()}')

# Save the trained model
model.save_pretrained('my_miniLM_model')


Epoch 1, Batch loss: 10.407831192016602
Epoch 2, Batch loss: 10.174388885498047
Epoch 3, Batch loss: 9.971243858337402
Epoch 4, Batch loss: 9.792301177978516
Epoch 5, Batch loss: 9.613380432128906
Epoch 6, Batch loss: 9.472424507141113
Epoch 7, Batch loss: 9.320964813232422
Epoch 8, Batch loss: 9.196343421936035
Epoch 9, Batch loss: 9.070536613464355
Epoch 10, Batch loss: 8.94410228729248
Epoch 11, Batch loss: 8.82468318939209
Epoch 12, Batch loss: 8.708112716674805
Epoch 13, Batch loss: 8.59731388092041
Epoch 14, Batch loss: 8.481447219848633
Epoch 15, Batch loss: 8.37304973602295
Epoch 16, Batch loss: 8.262899398803711
Epoch 17, Batch loss: 8.167189598083496
Epoch 18, Batch loss: 8.059585571289062
Epoch 19, Batch loss: 7.965031623840332
Epoch 20, Batch loss: 7.8821916580200195
Epoch 21, Batch loss: 7.777637004852295
Epoch 22, Batch loss: 7.698150157928467
Epoch 23, Batch loss: 7.615810871124268
Epoch 24, Batch loss: 7.536231517791748
Epoch 25, Batch loss: 7.452248573303223
Epoch 26, 

In [ ]:
import torch
from transformers import BertTokenizer, BertLMHeadModel, pipeline

# Load the trained MiniLM model
model = BertLMHeadModel.from_pretrained('my_miniLM_model')

# Load the tokenizer
# tokenizer = BertTokenizer.from_pretrained('path/to/your/trained/model')

# Generate text samples using the model
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)
samples = generator('Your prompt here', max_length=100, num_return_sequences=1)  # Set num_return_sequences to 1

# Print generated samples
for i, sample in enumerate(samples):
    print(f"Sample {i + 1}: {sample['generated_text']}")


OSError: ignored